In [1]:
import os
import pandas as pd
import re
import sqlite3
from hashlib import blake2b
from importlib import reload

In [2]:
os.chdir('../')

In [3]:
from src import extract as ex
from src import parse as pr
from src import normalise as nm

## Prepare Data

In [4]:
# load data
df = pd.read_csv("data/raw/PPR-ALL.csv", encoding='latin-1')
# create uuid
df = df.rename(columns={
    'Date of Sale (dd/mm/yyyy)': 'date',
    'Address': 'address',
    'Postal Code': 'postcode',
    'County': 'county',
    'Price (\x80)': 'price',
    'Not Full Market Price': 'not_full_market_price',
    'VAT Exclusive': 'vat_exclusive',
    'Property Size Description': 'floor_area',
    'Description of Property': 'construction',
})

# create uuid for each
for row in df.copy().itertuples():
    # hash_id_string = f"{row.date}{row.address}{row.price}"
    hash_id_string = f"{row.date}{row.address}{row.postcode}{row.county}{row.price}{row.not_full_market_price}{row.vat_exclusive}{row.floor_area}{row.construction}"
    df.at[row.Index, 'uuid'] = blake2b(bytes(hash_id_string, "utf-8"), digest_size=5).hexdigest()

# remove duplicates
df.drop_duplicates(inplace=True)

df.set_index('uuid', inplace=True)


/Users/br20069521/Desktop/Date-Dump/21-03-07/code_scratchpad/blog/parsing-the-property-price-register-part-1/venv/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [5]:
db_url = "data/database.db"
conn = sqlite3.connect(db_url)
cur = conn.cursor()
df.to_sql('ppr_all', conn)
cur.close()
conn.close()

### Create tempoary data table

In [6]:
conn = sqlite3.connect('data/database.db')
df = pd.read_sql_query('SELECT * FROM ppr_all', conn)
df.set_index('uuid', inplace=True)

df = df.drop(["date","price","not_full_market_price","construction","floor_area","vat_exclusive"], axis=1)
df.insert(3, 'ex_eircode', None)
df.insert(3, 'ex_county', None)
df.insert(3, 'ex_postcode', None)
df.insert(3, 'address_mod', None)
df.insert(3, 'xx', None)
df.address_mod = df.address.str.lower()


cur = conn.cursor()
cur.execute(f"DROP TABLE IF EXISTS temp;")
df.to_sql('temp', conn)
cur.close()
conn.close()

In [7]:
df

,address,postcode,county,xx,address_mod,ex_postcode,ex_county,ex_eircode
uuid,,,,,,,,
b306a21836,"5 Braemor Drive, Churchtown, Co.Dublin",None,Dublin,None,"5 braemor drive, churchtown, co.dublin",None,None,None
42e8eed8ad,"134 Ashewood Walk, Summerhill Lane, Portlaoise",None,Laois,None,"134 ashewood walk, summerhill lane, portlaoise",None,None,None
7b08b5e03d,"1 Meadow Avenue, Dundrum, Dublin 14",None,Dublin,None,"1 meadow avenue, dundrum, dublin 14",None,None,None
2352e180f5,"1 The Haven, Mornington",None,Meath,None,"1 the haven, mornington",None,None,None
cbac7ce7bd,"11 Melville Heights, Kilkenny",None,Kilkenny,None,"11 melville heights, kilkenny",None,None,None
...,...,...,...,...,...,...,...,...
b39bcc2471,"CROSSNEEN, GRAIGUECULLEN, CARLOW",None,Laois,None,"crossneen, graiguecullen, carlow",None,None,None
b65b929588,"GORTADERRA, SCARRIFF, CLARE",None,Clare,None,"gortaderra, scarriff, clare",None,None,None
3e02f239ba,"HALSEYRATH, DUNCORMICK, WEXFORD",None,Wexford,None,"halseyrath, duncormick, wexford",None,None,None


---

## Eirccode Extraction

In [8]:
conn = sqlite3.connect('data/database.db')
df = pd.read_sql_query('SELECT * FROM temp;', conn)
df.set_index('uuid', inplace=True)
df

,address,postcode,county,xx,address_mod,ex_postcode,ex_county,ex_eircode
uuid,,,,,,,,
b306a21836,"5 Braemor Drive, Churchtown, Co.Dublin",None,Dublin,None,"5 braemor drive, churchtown, co.dublin",None,None,None
42e8eed8ad,"134 Ashewood Walk, Summerhill Lane, Portlaoise",None,Laois,None,"134 ashewood walk, summerhill lane, portlaoise",None,None,None
7b08b5e03d,"1 Meadow Avenue, Dundrum, Dublin 14",None,Dublin,None,"1 meadow avenue, dundrum, dublin 14",None,None,None
2352e180f5,"1 The Haven, Mornington",None,Meath,None,"1 the haven, mornington",None,None,None
cbac7ce7bd,"11 Melville Heights, Kilkenny",None,Kilkenny,None,"11 melville heights, kilkenny",None,None,None
...,...,...,...,...,...,...,...,...
b39bcc2471,"CROSSNEEN, GRAIGUECULLEN, CARLOW",None,Laois,None,"crossneen, graiguecullen, carlow",None,None,None
b65b929588,"GORTADERRA, SCARRIFF, CLARE",None,Clare,None,"gortaderra, scarriff, clare",None,None,None
3e02f239ba,"HALSEYRATH, DUNCORMICK, WEXFORD",None,Wexford,None,"halseyrath, duncormick, wexford",None,None,None


In [9]:
for row in df.copy().itertuples():
    eircode = ex.extract_eircode_from_end_of_address(row.address_mod)
    if eircode is not None:
        df.at[row.Index, 'ex_eircode'] = eircode
        df.at[row.Index, 'address_mod'] = pr.parse_address_of_eircode(row.address_mod)

In [10]:
cur = conn.cursor()
cur.execute(f"DROP TABLE IF EXISTS temp;")
df.to_sql('temp', conn)
cur.close()
conn.close()

## Dublin Postcode Extraction

In [11]:
conn = sqlite3.connect('data/database.db')
df = pd.read_sql_query('SELECT * FROM temp;', conn)
df.set_index('uuid', inplace=True)
df.address_mod = df.address_mod.apply(nm.normalise_dublin_postcode)

for row in df.copy().itertuples():
    postcode = ex.extract_dublin_postcode_from_address(row.address_mod)
    if postcode is not None:
        df.at[row.Index, 'ex_postcode'] = postcode
        df.at[row.Index, 'address_mod'] = pr.parse_address_of_dublin_postcode(row.address_mod)
        
cur = conn.cursor()
cur.execute(f"DROP TABLE IF EXISTS temp;")
df.to_sql('temp', conn)
cur.close()
conn.close()

## County Extraction

In [12]:
conn = sqlite3.connect('data/database.db')
df = pd.read_sql_query('SELECT * FROM temp;', conn)
df.set_index('uuid', inplace=True)

df.address_mod = df.address_mod.apply(nm.normalise_county)
for row in df.copy().itertuples():
    county = ex.extract_county_from_address(row.address_mod)
    if county is not None:
        df.at[row.Index, 'ex_county'] = county
        df.at[row.Index, 'address_mod'] = pr.parse_address_of_county(row.address_mod)
cur = conn.cursor()
cur.execute(f"DROP TABLE IF EXISTS temp;")
df.to_sql('temp', conn)
cur.close()
conn.close()

## Combine with Origional Dataset

In [31]:
conn = sqlite3.connect('data/database.db')
df_orig = pd.read_sql_query('SELECT * FROM ppr_all;', conn)
conn.close()
df_orig.set_index('uuid', inplace=True)
df_orig.insert(4, 'eircode', None)
df_orig.head()

,date,address,postcode,county,eircode,price,not_full_market_price,vat_exclusive,construction,floor_area
uuid,,,,,,,,,,
b306a21836,01/01/2010,"5 Braemor Drive, Churchtown, Co.Dublin",None,Dublin,None,"343,000.00",No,No,Second-Hand Dwelling house /Apartment,None
42e8eed8ad,03/01/2010,"134 Ashewood Walk, Summerhill Lane, Portlaoise",None,Laois,None,"185,000.00",No,Yes,New Dwelling house /Apartment,greater than or equal to 38 sq metres and less...
7b08b5e03d,04/01/2010,"1 Meadow Avenue, Dundrum, Dublin 14",None,Dublin,None,"438,500.00",No,No,Second-Hand Dwelling house /Apartment,None
2352e180f5,04/01/2010,"1 The Haven, Mornington",None,Meath,None,"400,000.00",No,No,Second-Hand Dwelling house /Apartment,None
cbac7ce7bd,04/01/2010,"11 Melville Heights, Kilkenny",None,Kilkenny,None,"160,000.00",No,No,Second-Hand Dwelling house /Apartment,None


### Combine Eircode

In [32]:
conn = sqlite3.connect('data/database.db')
df = pd.read_sql_query('SELECT * FROM temp WHERE ex_eircode NOT NULL;', conn)
conn.close()
df.set_index('uuid', inplace=True)
df.head()


,address,postcode,county,xx,address_mod,ex_postcode,ex_county,ex_eircode
uuid,,,,,,,,
ccf905f559,"2 SLANELOUGH, ASHFORD, CO WICKLOW A67 ED73",None,Wicklow,xx,"2 slanelough, ashford",None,Wicklow,A67 ED73
f649ac5a17,"PARSONS GATES, BELMONT R0AD, CASTLECONNELL V94...",None,Limerick,xx,"parsons gates, belmont r0ad, castleconnell",None,None,V94 VPW3
be0c8afe5b,"7 Montpellier Terrace, Galway, H91 C2DN",None,Galway,xx,7 montpellier terrace,None,Galway,H91 C2DN
3abb465dd9,"21 Harbour View, The Square, Clifden H71 AH59",None,Galway,xx,"21 harbour view, the square, clifden",None,None,H71 AH59
bbad706bf6,"Apartment 19 Desart Court, New Street, Kilkenn...",None,Kilkenny,xx,"apartment 19 desart court, new street",None,Kilkenny,R95 X050


In [33]:
for row in df.copy().itertuples():
    df_orig.at[row.Index, 'address'] = row.address_mod.title()
    df_orig.at[row.Index, 'eircode'] = row.ex_eircode

### Combine Dublin Postcode

In [34]:
conn = sqlite3.connect('data/database.db')
df = pd.read_sql_query('SELECT * FROM temp WHERE ex_postcode NOT NULL;', conn)
conn.close()
df.set_index('uuid', inplace=True)
df.head()

,address,postcode,county,xx,address_mod,ex_postcode,ex_county,ex_eircode
uuid,,,,,,,,
7b08b5e03d,"1 Meadow Avenue, Dundrum, Dublin 14",None,Dublin,None,"1 meadow avenue, dundrum",Dublin 14,None,None
6b35e03f73,"206 Philipsburgh Avenue, Marino, Dublin 3",None,Dublin,None,"206 philipsburgh avenue, marino",Dublin 3,None,None
a7cf1b1d16,"22 Laverna Way, Castleknock, Dublin 15",None,Dublin,None,"22 laverna way, castleknock",Dublin 15,None,None
dc92e8c267,"33 RAGIAN ROAD, BALLSBRIDGE, DUBLIN 4",None,Dublin,None,"33 ragian road, ballsbridge",Dublin 4,None,None
6d10ff8567,"5 Clancy Road, Finglas, Dublin 11",None,Dublin,None,"5 clancy road, finglas",Dublin 11,None,None


In [35]:
for row in df.copy().itertuples():
    ex_postcode = row.ex_postcode
    postcode = row.postcode
    county = row.county
    if postcode is None:
        df_orig.at[row.Index, 'postcode'] = row.ex_postcode
        if county != 'Dublin':
            df_orig.at[row.Index, 'county'] = f"{county}, (ex_postcode: Dublin)"
    else:
        if postcode.title() != ex_postcode:
            df_orig.at[row.Index, 'postcode'] = f"{postcode}, (ex_postcode: {ex_postcode})"
        if county != 'Dublin':
            df_orig.at[row.Index, 'county'] = f"{county}, (ex_postcode: Dublin)"
    df_orig.at[row.Index, 'address'] = row.address_mod.title()

### Combine County

In [36]:
conn = sqlite3.connect('data/database.db')
df = pd.read_sql_query('SELECT * FROM temp WHERE ex_county NOT NULL;', conn)
conn.close()
df.set_index('uuid', inplace=True)
df.head()

,address,postcode,county,xx,address_mod,ex_postcode,ex_county,ex_eircode
uuid,,,,,,,,
b306a21836,"5 Braemor Drive, Churchtown, Co.Dublin",None,Dublin,None,"5 braemor drive, churchtown",None,Dublin,None
cbac7ce7bd,"11 Melville Heights, Kilkenny",None,Kilkenny,None,11 melville heights,None,Kilkenny,None
42679e4628,"13 Oakleigh Wood, Dooradoyle, Limerick",None,Limerick,None,"13 oakleigh wood, dooradoyle",None,Limerick,None
decc394ad1,"25 Lavallin Drive, Whitechurch, Cork",None,Cork,None,"25 lavallin drive, whitechurch",None,Cork,None
347e854a9b,"44 Allen Park Road, Stillorgan, County Dublin",None,Dublin,None,"44 allen park road, stillorgan",None,Dublin,None


In [37]:
for row in df.copy().itertuples():
    ex_county = row.ex_county
    county = row.county
    if county != ex_county:
        df_orig.at[row.Index, 'county'] = f"{county}, (ex_county: {ex_county})"
    df_orig.at[row.Index, 'address'] = row.address_mod.title()

In [38]:
conn = sqlite3.connect('data/database.db')
cur = conn.cursor()
cur.execute(f"DROP TABLE IF EXISTS ppr_all_mod;")
df_orig.to_sql('ppr_all_mod', conn)
cur.close()
conn.close()